In [1]:
import sqlalchemy
import pandas as pd
import numpy as np
import urllib
import math
import uuid
import time
from datetime import date # for datediff calc
from datetime import datetime # for remit dates
import os

# sqlalchemy imports
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
from sqlalchemy import inspect
from sqlalchemy.pool import NullPool

In [2]:
# Step1. get loan #s from spd buyout table
# container --> has loan #s. need to change this to query instead of csv pull - DONE
# connect to server --> connect_server(my_server,my_database) - DONE
# Step 1a.? create sql tables --> create_sql_tables(container,sql_conn_mls) - DONE

# Step 2a. create temp table for qrm - DONE
# Step 2. create df for qrm_pulsar - DONE
# Step 3. create df for spd pools - DONE
# Step 4. create df for transaction pandi and or investor accounting table

## Define Variables

In [3]:
login = os.getlogin()
date_string = time.strftime("%Y%m%d", time.localtime()) # for use in timestamp for export files

In [4]:
date_string

'20241009'

## Define SQL Functions and Variables

In [5]:
def get_loan_numbers_to_create_temp_table(path_import, file_name):
    '''
    Reads eligibility csv file and converts into dictionary to create temp table
    Make sure file_name parameter includes the csv or xlsx extension. Script will check for it.
    Make sure to use csv, and loan number column titled 'LoanId'
    End result is a data container that contains tempTableName and input as a dataframe of the loan list
    '''

    # check for csv or xlsx in filename
    if file_name[-3:] == 'csv':
        df = pd.read_csv(path_import + '\\' + file_name)['LoanId'].to_frame(name=None)
    else:
        df = pd.read_excel(path_import + '\\' + file_name)['LoanId'].to_frame(name=None)

    packageInfo = {'tempTableName': '##PBOTracker', 'inputs': df}
    
    return packageInfo


In [6]:
def connect_server(serverName,dataBase):
    '''
    Return a connection to a MS SQL Server using sqlalchemy,
        this is an instance and will stay alive as long as you dont close it
        input parameters serverName and dataBase are strings
    '''
    
    conn_str = urllib.parse.quote_plus(r'DRIVER={ODBC Driver 17 for SQL Server};SERVER=' +
                                       serverName + 
                                       r';DATABASE=' +
                                       dataBase + 
                                       r';TRUSTED_CONNECTION=yes')
    conn = 'mssql+pyodbc:///?odbc_connect={}'.format(conn_str)
    engine = sqlalchemy.create_engine(conn)
    connection = engine.connect()
    return connection


In [7]:
# Step 1: get loan numbers for container
my_server_fc1 = 'SRV-SQLFC-1'
my_database_fc1 = 'SPD'
my_path_sql = 'M:\Capital Markets\PIPE\EBO\SQL Scripts\Adhoc Queries'
my_filename_sql = 'get_ebo_buyout_loannumbers.sql'

In [8]:
def create_dataframe_from_sql_query(server_name, database_name, query_path, query_name):
    '''
    Creates dataframe from a SQL query. See below for parameter descriptions
        server_name: name of server for query
        database_name: name of database for query
        query_path: path where sql query resides on shared drive
        query_name: name of sql query to use
    '''
    
    conn = connect_server(server_name, database_name)
   
    df = pd.read_sql_query(get_sql_query(query_path,query_name),
                                     con = conn)
    conn.close()
    return df

In [9]:
# create_dataframe_from_sql_query(container, my_server, my_database, query_path, query_name_oterm)

In [10]:
# helper function to get sql query on shared drive
def get_sql_query(path_import_sql, filename_sql):
    '''
    Filename: name of sql file
    returns the sql file contents as a string
    '''
    # opens file in read mode
    sqlFile = open(f'{path_import_sql}\\{filename_sql}', 'r') 
    
    # read file into variable
    query = sqlFile.read()
    return query

## Get Loannums Using SQL

In [11]:
df_loannums = create_dataframe_from_sql_query(my_server_fc1, my_database_fc1, my_path_sql, my_filename_sql)

In [12]:
df_loannums.head()

,LoanId
0,7.000730e+09
1,1.004431e+09
2,1.005020e+09
3,1.000294e+09
4,8.021949e+09


In [13]:
container = {'tempTableName': '##PBOTracker', 'inputs': df_loannums}

In [14]:
# need to create temp table for qrm, for spd on fc1 just make the initial table, 

In [15]:
def create_sql_tables(packageInfo,conn):
    """
    Creates a temp table on the given connection.
    packageInfo parameter should include tempTableName and dataframe consisting of LoanIds
    Temp table contains loan ids and UPB from the given data date.
    Note: Loan input needs to have loan #'s column as 'LoanId' otherwise it will return an empty TempTable
    """
    # Get meta data
    metadata = MetaData()

    # Define SQL Table
    populationTable = Table(packageInfo['tempTableName'],metadata, 
        Column('LoanId', String, primary_key=False)
    )
   
    # Create the table
    populationTable.create(conn, checkfirst=True)
    print(f"Temp table - {packageInfo['tempTableName']} created.")

    # Populate the table
    data = packageInfo["inputs"].to_dict('records')
    rows = 1000
    nIter = int(math.ceil(len(data)/rows))
    for n in range(1,nIter+1,1):
        conn.execute(populationTable.insert().values(data[(n-1) * rows : n * rows]))
    
    conn.close()
    return populationTable

## Get QRM Redelivery Data (qrm_pulsar)

In [16]:
# Step 2
# create qrm_pulsar dataframe - see if this openquery works
# need to check pmsr to see what it does for multiple periods
my_server_qrm = 'AWWAPULSSQLDP01'
my_database_qrm = 'qrm_pulsar'
my_path_sql_qrm = 'M:\Capital Markets\PIPE\EBO\SQL Scripts\Adhoc Queries'
my_filename_sql_qrm = 'QRM_pipe_PL.sql'
qrm_conn = connect_server(my_server_qrm,my_database_qrm)
create_sql_tables(container,qrm_conn)

Temp table - ##PBOTracker created.


Table('##PBOTracker', MetaData(), Column('LoanId', String(), table=<##PBOTracker>), schema=None)

In [17]:
qrm_conn.close()

In [18]:
df_qrm = create_dataframe_from_sql_query(my_server_qrm, my_database_qrm, my_path_sql_qrm, my_filename_sql_qrm)

In [19]:
fc1_conn = connect_server(my_server_fc1,my_database_fc1)
create_sql_tables(container,fc1_conn)

Temp table - ##PBOTracker created.


Table('##PBOTracker', MetaData(), Column('LoanId', String(), table=<##PBOTracker>), schema=None)

In [20]:
fc1_conn.close()

In [21]:
df_qrm.head()

,Loan Number,Face Amount,Issue Date Balance,YearMonth,Buy Price,Shared Econ,repurchase_date_calc,investor_redelivery_premium_pct,investor_redelivery_premium_dlr,penny_redelivery_premium_pct,penny_redelivery_premium_dlr,Pool Num,Management,Pool Status,Type Dimension,Loan Type,settled date,settled price,EBO_Investor,Path_Reperf
0,1000034167,202.18975,201.81871,202002,100.000,0,2020-02-11,1.00000,202189.750000,0.02605,5257.377395,BR3339,EBO,Settled,CL,F30MOD,2020-02-21,102.605469,PNMAC,Mod
1,1000034173,316.73414,316.19527,202202,100.275,50,2022-01-25,0.00275,871.018885,0.00275,871.018885,CJ6303,EBO,Settled,CL,F30MOD,2022-02-03,100.816406,MassMutual,Mod
2,1000034185,473.12430,472.33205,202112,100.000,0,2021-12-18,1.00000,473124.300000,0.03797,17934.447938,CJ6164,EBO,Settled,CL,F30MOD,2021-12-27,103.796875,PNMAC,Mod
3,1000034185,471.48519,471.48519,202306,100.000,0,2023-05-28,1.00000,471485.190000,0.01121,5285.348980,CV2547,EBO,Settled,CL,F30MOD,2023-06-06,101.121094,PNMAC,Mod
4,1000034250,153.87552,153.58475,202007,100.000,0,2020-07-19,1.00000,153875.520000,0.07523,11554.180743,BW6275,EBO,Settled,CL,F30MOD,2020-07-27,107.523438,PNMAC,Mod


## Get QRM Historical Market Data (qrm_pulsar)

In [22]:
# Step 2
# create qrm_pulsar dataframe - see if this openquery works
# need to check pmsr to see what it does for multiple periods
# my_server_qrm = 'AWWAPULSSQLDP01' # already defined above
# my_database_qrm = 'qrm_pulsar' # already defined above
# my_path_sql_qrm = 'M:\Capital Markets\PIPE\EBO\SQL Scripts\Adhoc Queries' # already defined above
my_filename_sql_qrm_market = 'QRM_HistoricalMarket_prices_eom.sql'
qrm_conn_mkt = connect_server(my_server_qrm,my_database_qrm)
# create_sql_tables(container,qrm_conn_mkt) # <<<< might have to change this

In [23]:
df_qrm_market = create_dataframe_from_sql_query(my_server_qrm, my_database_qrm, my_path_sql_qrm, my_filename_sql_qrm_market)

In [24]:
df_qrm_market.meperiod_qrm_market = df_qrm_market.meperiod_qrm_market.astype('int64') # for when merging with cte_ebo dataframe # meperiod is int64

## Get Main EBO Trade Data (FC1)

In [25]:
# Step 3. create df for spd pools
my_server_fc1 = 'SRV-SQLFC-1'
my_database_fc1 = 'SPD'
my_path_sql_fc1 = 'M:\Capital Markets\PIPE\EBO\SQL Scripts\Adhoc Queries'
my_filename_sql_ebo = 'ebo_program_cte_fc1.sql'
df_ebo = create_dataframe_from_sql_query(my_server_fc1, my_database_fc1, my_path_sql_fc1, my_filename_sql_ebo)

In [26]:
df_ebo.head()

,PrimeKey_pbo,SaleMonth,LoanId,Investor_pbosale,PBOSale_SalePassThru,PBOSale_SaleUPB_Orig,GoSSplit_pbosale,MEPeriod_pbo,trade_period,wire_date_remit,...,PBOSale_SalePassThru2,PBOSale_SaleDate,PBOSale_IntPaidToDate,me_tradeperiod_0,LoanType_2,path_pif,path_liq,investor_pbosale_vector_pivot,clean_and_current_4mo,clean_and_current_7mo
0,1.000155e+13,202203,1000154904,NB,0.04,176416.66,0.5,202202,0,2022-03-15,...,0.04,2022-03-01,2021-11-01,202202,FHA,NULL,NULL,NB,0,0
1,1.000155e+13,202203,1000154904,NB,0.04,176416.66,0.5,202203,1,2022-04-15,...,NaN,None,None,None,None,NULL,NULL,NB,0,0
2,1.000155e+13,202203,1000154904,NB,0.04,176416.66,0.5,202204,2,2022-05-15,...,NaN,None,None,None,None,NULL,NULL,NB,0,0
3,1.000155e+13,202203,1000154904,NB,0.04,176416.66,0.5,202205,3,2022-06-15,...,NaN,None,None,None,None,NULL,NULL,NB,0,0
4,1.000155e+13,202203,1000154904,NB,0.04,176416.66,0.5,202206,4,2022-07-15,...,NaN,None,None,None,None,NULL,NULL,NB,0,0


In [27]:
df_ebo.shape

(13878, 96)

In [28]:
df_qrm.shape

(165402, 20)

In [29]:
# distinct loannums from ebo program
df_loannums.shape

(96004, 1)

In [30]:
df_ebo.PrimeKey_pbo = df_ebo.PrimeKey_pbo.astype('int64')

In [31]:
df_ebo.PrimeKey_pbo = df_ebo.PrimeKey_pbo.astype('int64')

In [32]:
df_ebo.head()

,PrimeKey_pbo,SaleMonth,LoanId,Investor_pbosale,PBOSale_SalePassThru,PBOSale_SaleUPB_Orig,GoSSplit_pbosale,MEPeriod_pbo,trade_period,wire_date_remit,...,PBOSale_SalePassThru2,PBOSale_SaleDate,PBOSale_IntPaidToDate,me_tradeperiod_0,LoanType_2,path_pif,path_liq,investor_pbosale_vector_pivot,clean_and_current_4mo,clean_and_current_7mo
0,10001549042203,202203,1000154904,NB,0.04,176416.66,0.5,202202,0,2022-03-15,...,0.04,2022-03-01,2021-11-01,202202,FHA,NULL,NULL,NB,0,0
1,10001549042203,202203,1000154904,NB,0.04,176416.66,0.5,202203,1,2022-04-15,...,NaN,None,None,None,None,NULL,NULL,NB,0,0
2,10001549042203,202203,1000154904,NB,0.04,176416.66,0.5,202204,2,2022-05-15,...,NaN,None,None,None,None,NULL,NULL,NB,0,0
3,10001549042203,202203,1000154904,NB,0.04,176416.66,0.5,202205,3,2022-06-15,...,NaN,None,None,None,None,NULL,NULL,NB,0,0
4,10001549042203,202203,1000154904,NB,0.04,176416.66,0.5,202206,4,2022-07-15,...,NaN,None,None,None,None,NULL,NULL,NB,0,0


In [33]:
df_ebo.dtypes

PrimeKey_pbo                       int64
SaleMonth                          int64
LoanId                             int64
Investor_pbosale                  object
PBOSale_SalePassThru             float64
                                  ...   
path_pif                          object
path_liq                          object
investor_pbosale_vector_pivot     object
clean_and_current_4mo              int64
clean_and_current_7mo              int64
Length: 96, dtype: object

In [34]:
# change YearMonth to int64 for merge step
df_qrm.YearMonth = df_qrm.YearMonth.astype('int64')

In [35]:
df_qrm.dtypes

Loan Number                         object
Face Amount                        float64
Issue Date Balance                 float64
YearMonth                            int64
Buy Price                          float64
Shared Econ                          int64
repurchase_date_calc                object
investor_redelivery_premium_pct    float64
investor_redelivery_premium_dlr    float64
penny_redelivery_premium_pct       float64
penny_redelivery_premium_dlr       float64
Pool Num                            object
Management                          object
Pool Status                         object
Type Dimension                      object
Loan Type                           object
settled date                        object
settled price                      float64
EBO_Investor                        object
Path_Reperf                         object
dtype: object

In [36]:
df_qrm['Loan Number'] = df_qrm['Loan Number'].astype('int64')

## Merge Trade and QRM Redelivery Data

In [37]:
# new_df = pd.merge(A_df, B_df,  how='left', left_on=['A_c1','c2'], right_on = ['B_c1','c2'])
new_df = pd.merge(df_ebo, df_qrm,  how='left', left_on=['LoanId','MEPeriod_pbo'], right_on = ['Loan Number','YearMonth'])

## Merge Trade and QRM Historical Data

In [38]:
new_df.MEPeriod_pbo.dtype

dtype('int64')

In [39]:
df_qrm_market.meperiod_qrm_market.dtype

dtype('int64')

In [40]:
# df_qrm_market
new_df = pd.merge(new_df, df_qrm_market, how='left', left_on=['MEPeriod_pbo'], right_on = ['meperiod_qrm_market'])

### new_df Head, Columns, Dtypes, and Shape

In [41]:
new_df.head()

,PrimeKey_pbo,SaleMonth,LoanId,Investor_pbosale,PBOSale_SalePassThru,PBOSale_SaleUPB_Orig,GoSSplit_pbosale,MEPeriod_pbo,trade_period,wire_date_remit,...,qrm_GN30_40_px_eom,qrm_GN30_45_px_eom,qrm_GN30_50_px_eom,qrm_GN30_55_px_eom,qrm_GN30_60_px_eom,qrm_GN30_65_px_eom,qrm_GN30_70_px_eom,qrm_UST_10Y_px_eom,qrm_PMMS_px_eom,meperiod_qrm_market
0,10001549042203,202203,1000154904,NB,0.04,176416.66,0.5,202202,0,2022-03-15,...,104.075997,104.767253,105.818539,0.000000,0.000000,0.0,0.0,1.842570,3.757634,202202
1,10001549042203,202203,1000154904,NB,0.04,176416.66,0.5,202203,1,2022-04-15,...,102.293935,103.770583,103.836304,0.000000,0.000000,0.0,0.0,2.327710,4.569074,202203
2,10001549042203,202203,1000154904,NB,0.04,176416.66,0.5,202204,2,2022-05-15,...,100.437188,102.307482,103.607178,103.607178,NaN,0.0,0.0,2.886340,5.139656,202204
3,10001549042203,202203,1000154904,NB,0.04,176416.66,0.5,202205,3,2022-06-15,...,101.211005,102.469217,103.533472,103.533472,103.533472,0.0,0.0,2.833471,5.088799,202205
4,10001549042203,202203,1000154904,NB,0.04,176416.66,0.5,202206,4,2022-07-15,...,99.832463,101.813462,102.821310,103.853298,102.821310,0.0,0.0,2.956478,5.550806,202206


In [42]:
new_df.columns

Index(['PrimeKey_pbo', 'SaleMonth', 'LoanId', 'Investor_pbosale',
       'PBOSale_SalePassThru', 'PBOSale_SaleUPB_Orig', 'GoSSplit_pbosale',
       'MEPeriod_pbo', 'trade_period', 'wire_date_remit',
       ...
       'qrm_GN30_40_px_eom', 'qrm_GN30_45_px_eom', 'qrm_GN30_50_px_eom',
       'qrm_GN30_55_px_eom', 'qrm_GN30_60_px_eom', 'qrm_GN30_65_px_eom',
       'qrm_GN30_70_px_eom', 'qrm_UST_10Y_px_eom', 'qrm_PMMS_px_eom',
       'meperiod_qrm_market'],
      dtype='object', length=133)

In [43]:
new_df.dtypes

PrimeKey_pbo              int64
SaleMonth                 int64
LoanId                    int64
Investor_pbosale         object
PBOSale_SalePassThru    float64
                         ...   
qrm_GN30_65_px_eom      float64
qrm_GN30_70_px_eom      float64
qrm_UST_10Y_px_eom      float64
qrm_PMMS_px_eom         float64
meperiod_qrm_market       int64
Length: 133, dtype: object

In [44]:
new_df.shape

(13878, 133)

## Export raw new_df (comment out if not needed)

In [45]:
new_df.to_csv(f'C:\\Users\\{login}\\Desktop\\new_df_raw_{date_string}_202203SaleMonth.csv')

## Grab raw data from csv - Remove if already pulling SQL

## Grab raw data from csv

In [46]:
# new_df = pd.read_csv(r'C:\Users\jboyce\Desktop\new_df_raw_20240718_202101SaleMonth.csv')
# VVVV PENDING USE THIS VERSION ONCE TESTING DONE VVVV
# new_df = pd.read_csv(f'C:\\Users\\{login}\\Desktop\\new_df_raw_{date_string}_202101SaleMonth.csv')

In [47]:
# new_df=new_df.fillna(0)

In [48]:
# new_df = new_df.loc[:, ~new_df.columns.str.contains('^Unnamed')]

In [49]:
# new_df.head()

In [50]:
# new_df.shape

In [51]:
# print(np.unique(Investor_pbosale))
print(new_df['Investor_pbosale'].unique())

['NB' 'MM' 'TO']


In [52]:
# FINAL COPY
def remove_extra_periods_backup20221025(data, 
                         column_to_filter='PrimeKey', 
                         column_to_target='trade_period'):
    """
    This function removes trade periods after loan is redelivered
    dataframe: df to start
    column_to_filter1: index-ish column to filter from (PrimeKey as default)
    column_to_target: column that has the periods to remove (trade_period as default)
    """
    
    # get list of values from the filtering column
    filter_list = data[column_to_filter].values
    
    # condense it to a unique list instead of repeating for each period
    filter_list_unique = np.unique(filter_list)
    
    trade_period_list = []
    
    # cycle through each filter and remove the desired periods
    for v in filter_list_unique:
        trade_period_list = data.loc[(data[column_to_filter] == v) & (data['Management'] == 'EBO')][column_to_target].values
        first_trade_period_0 = trade_period_list[0] # might have multiple redelivery periods. only need the 1st
        
        # delete data between these periods
        data.drop(data.loc[(data[column_to_filter] == v) & 
                           (data[column_to_target] > first_trade_period_0)].index, 
                 inplace=True)
        
    return data

## Function - Remove Extra Periods

In [53]:
# FINAL COPY - TAKES TOO LONG
def remove_extra_periods(data, 
                         column_to_filter='PrimeKey_pbo', 
                         column_to_target='trade_period'):
    """
    This function removes trade periods after loan is redelivered
    dataframe: df to start
    column_to_filter1: index-ish column to filter from (PrimeKey as default)
    column_to_target: column that has the periods to remove (trade_period as default)
    """
    # attempting to fix "IndexError: index 0 is out of bounds for axis 0 with size 0"
    # https://stackoverflow.com/questions/41492288/what-does-index-0-is-out-of-bounds-for-axis-0-with-size-0-mean
    # looks like I need to add a try-except block to account for size 0

    
    # get list of values from the filtering column
    filter_list = data[column_to_filter].values
    
    # condense it to a unique list instead of repeating for each period
    filter_list_unique = np.unique(filter_list)
    
    trade_period_list = []
    
    # cycle through each filter and remove the desired periods
    for v in filter_list_unique:
        trade_period_list = data.loc[(data[column_to_filter] == v) & (data['Management'] == 'EBO')][column_to_target].values
        
        # check to see if loan was ever repurchased (list len >0)
        if len(trade_period_list) >0:
            # ADD BACK AFTER TEST
            first_trade_period_0 = trade_period_list[0] # might have multiple redelivery periods. only need the 1st
            data.drop(data.loc[(data[column_to_filter] == v)
                               & (data[column_to_target] > first_trade_period_0)].index,
                      inplace=True)
        else:
            pass
            
        
        
        # ADD CHECK FOR LEN FOR first_trade_period_0
        # CHECK FOR EMPTY LIST
        # print(first_trade_period_0) # test
        # print(trade_period_list) # test
        # print(len(trade_period_list)) # test
        
        # delete data between these periods
        # ADD THIS BOTTOM SECTION BACK AFTER 0 SIZE TEST
        # data.drop(data.loc[(data[column_to_filter] == v) & 
        #                   (data[column_to_target] > first_trade_period_0)].index, 
        #         inplace=True)
        
    return data

## Function - Filter Out Extra Periods

In [54]:
# FINAL COPY - MIGHT NOT BE NEEDED AFTER SQL FILTER UPDATE
def filter_out_extra_periods(data, 
                         column_to_filter='PrimeKey_pbo', 
                         column_to_target='trade_period'):
    """
    This function removes trade periods after loan is redelivered
    dataframe: df to start
    column_to_filter1: index-ish column to filter from (PrimeKey as default)
    column_to_target: column that has the periods to remove (trade_period as default)
    """
    # attempting to fix "IndexError: index 0 is out of bounds for axis 0 with size 0"
    # https://stackoverflow.com/questions/41492288/what-does-index-0-is-out-of-bounds-for-axis-0-with-size-0-mean
    # looks like I need to add a try-except block to account for size 0

    
    # get list of values from the filtering column
    filter_list = data[column_to_filter].values
    
    # condense it to a unique list instead of repeating for each period
    filter_list_unique = np.unique(filter_list)
    
    trade_period_list = []
    data['trade_period_to_filter_out'] = 0
    
    # cycle through each filter and remove the desired periods
    for v in filter_list_unique:
        trade_period_list = data.loc[(data[column_to_filter] == v) & (data['Management'] == 'EBO')][column_to_target].values
        
        # check to see if loan was ever repurchased (list len >0)
        if len(trade_period_list) >0:
            # ADD BACK AFTER TEST
            first_trade_period_0 = trade_period_list[0] # might have multiple redelivery periods. only need the 1st
            # VVVV change this section to filter rather than drop
            # data.drop(data.loc[(data[column_to_filter] == v)
            #                    & (data[column_to_target] > first_trade_period_0)].index,
            #           inplace=True)
            # TRY - CREATE NEW COLUMN WITH 1s 0s (1s are trade periods > redelivery, filter new dataframe on 0)
            # data['Documentation Type'] = np.where((data['DOCUMENT'] >1), 'Low / No Doc', 'Full')
            # data['trade_period_to_filter_out'] = np.where((data[column_to_filter] == v) & (data[column_to_target] > first_trade_period_0), 1, 0)
    
        else:
            pass

    # filter out trade periods post-redelivery    
    # need to bring this outside the loop
    # might have to get the indexes for periods > redelivery
    # vvvv OLD SOLUTION 
    data.loc[(data[column_to_filter] == v) &
                            (data[column_to_target] > first_trade_period_0),
                            'trade_period_to_filter_out'] = 1

    data_filtered = data.loc[(data['trade_period_to_filter_out'] == 0)] # SettingWithCopyWarning
    # ^^^^ OLD SOLUTION
    # data.loc[
    #     (data[column_to_filter] == v) & (data[column_to_target] > first_trade_period_0),
    #     'trade_period_to_filter_out'
    #     ] = 1

    # data_filtered = data[(data['trade_period_to_filter_out'] == 0)]  # No need for .copy() now # this fixes the SettingWithCopyWarning
        
    return data_filtered

## Function - Get Min Redelivery-Eligible Period for PENDING Loans

In [55]:
# 
def get_min_pending_redelivery_eligible_period(data: pd.DataFrame, 
                         column_to_filter='PrimeKey_pbo', 
                         column_to_target='trade_period')->pd.DataFrame:
    """
    This function gets the minimum redelivery eligible period for loans that were 
    originally flagged as PENDING in the path_final flag
    
    dataframe: df to start
    column_to_filter1: index-ish column to filter from (PrimeKey as default)
    column_to_target: column that has the periods to remove (trade_period as default)
    """
    
    # get list of values from the filtering column
    filter_list = data[column_to_filter].values
    
    # condense it to a unique list instead of repeating for each period
    filter_list_unique = np.unique(filter_list)
    
    trade_period_dict = {}
    trade_period_list = []
    data['trade_period_min_pending_redeliverable_flag'] = 0 # <<< PENDING update this column
    
    # cycle through each filter and remove the desired periods
    for item in filter_list_unique:
        # trade_period_list = data.loc[((data[column_to_filter] == v) & 
        trade_period_pending_detail = data.loc[((data[column_to_filter] == item) & 
                                      (data['path_final'] == 'PENDING') & 
                                    #   (data['redelivery_elig_flag'] == 1))][column_to_target].values # <<<< PENDING maybe add min filter here; check filter used in excel template
                                      (data['redelivery_elig_flag'] == 1))][column_to_target].min()
        
        trade_period_dict.update({item: trade_period_pending_detail})

        #  add filter for
        # path_final = PENDING
        # or maybe try just finding min trade period given primekey == primekey and path_final == pending and redelivery_eligible == 1
        
        # check to see if loan was ever repurchased (list len >0)
        # if len(trade_period_list) >0:
        #     # ADD BACK AFTER TEST
        #     first_trade_period_0 = trade_period_list[0] # might have multiple redelivery periods. only need the 1st
        # else:
        #     pass

    # filter out trade periods post-redelivery    
    # need to bring this outside the loop
    # might have to get the indexes for periods > redelivery
    for i, (k, v) in enumerate(trade_period_dict.items()):
        data.loc[(data['PrimeKey_pbo']==k) & (data['trade_period']==v), 'trade_period_min_pending_redeliverable_flag']=1 # might have to use np.where
        # dataframe['terminal_path_flag'] = np.where(((dataframe['PrimeKey_pbo']==k) & (dataframe['trade_period']==v)),1,0)

    # vvvv OLD SOLUTION     
    # data.loc[(data[column_to_filter] == v) &
    #                         (data[column_to_target] > first_trade_period_0),
    #                         'trade_period_min_pending_redeliverable_flag'] = 1
    # data_filtered = data
        
    return data

In [56]:
# new_df.shape # shape before removing extra periods # (2112861, 103)

In [57]:
# new_df_filtered = filter_out_extra_periods(new_df) # UnboundLocalError: local variable 'first_trade_period_0' referenced before assignment
# probably need to get first_trade_period_0 back in scope

## Function - Days360

In [58]:
def days360Calc(start_date, end_date, method_eu=False):
    import calendar
#    start_day = x.year #maybe convert it to day string

    start_day = start_date.day
    start_month = start_date.month
    start_year = start_date.year
    end_day = end_date.day
    end_month = end_date.month
    end_year = end_date.year

    if (
        start_day == 31 or
        (
            method_eu is False and
            start_month == 2 and (
                start_day == 29 or (
                    start_day == 28 and
                    calendar.isleap(int(start_date.year)) is False # or calendar.isleap(year) or start_date.year.isleap() # was start_date.is_leap_year is False
                )
            )
        )
    ):
        start_day = 30

    if end_day == 31:
        if method_eu is False and start_day != 30:
            end_day = 1

            if end_month == 12:
                end_year += 1
                end_month = 1
            else:
                end_month += 1
        else:
            end_day = 30

    return (
        end_day + end_month * 30 + end_year * 360 -
        start_day - start_month * 30 - start_year * 360)


In [59]:
# df_test_trimmed['PBOSale_IntPaidToDate'] = pd.to_datetime(df_test_trimmed['PBOSale_IntPaidToDate'])

In [60]:
# df_test_trimmed['PBOSale_SaleDate'] = df_test_trimmed['PBOSale_SaleDate'].apply(lambda x: pd.to_datetime(str(x),format='%Y%m%d'))

In [61]:
# df_test_trimmed['PBOSale_SaleDate'] = settle_date
# NEED TO CONVERT SALE DATE FROM INT TO DATE

In [62]:
""" df_test_trimmed['DAYS360_DAYS'] = df_test_trimmed.apply(
    lambda row: days360Calc(row['PBOSale_IntPaidToDate'], row['PBOSale_SaleDate']), 
    axis=1)"""

" df_test_trimmed['DAYS360_DAYS'] = df_test_trimmed.apply(\n    lambda row: days360Calc(row['PBOSale_IntPaidToDate'], row['PBOSale_SaleDate']), \n    axis=1)"

In [63]:
new_df.EBO_Investor_y.unique()

array([nan, 'NexBank', 'MassMutual', 'Toxaway', 'PNMAC'], dtype=object)

In [64]:
new_df['Shared Econ'].unique()

array([nan, 50.,  0.])

## Filter Trade Populations for Analysis

In [65]:
# df_20220201_TO = new_df.loc[(new_df.Investor_pbosale.isin(['TO'])) & (new_df.SaleMonth.isin([202202]))]

In [66]:
# df_20220201_TO.head()

In [67]:
# df_20220201_TO.shape

## Consider creating FS function (e.g., datetime, days360, accrued interest, etc.)

## PENDING - FILTER OUT UNNECESSARY TRADE PERIODS FROM CSV - FILTERED CSV

In [68]:
# new_df_filtered = remove_extra_periods_optimized(new_df)
# new_df_filtered = filter_out_extra_periods(new_df)

In [69]:
# new_df_filtered.to_csv(r'C:\Users\jboyce\Desktop\new_df_filtered_20240618_202101SaleMonth.csv')

In [70]:
# new_df_filtered = pd.read_csv(r'C:\Users\jboyce\Desktop\new_df_filtered_20240618_202101SaleMonth.csv')

In [71]:
# new_df_filtered=new_df_filtered.fillna(0)

In [72]:
# new_df_filtered = new_df_filtered.loc[:, ~new_df_filtered.columns.str.contains('^Unnamed')]

In [73]:
# new_df_filtered.head()

In [74]:
# new_df_filtered.shape

## Function - Convert to Datetime

In [75]:
def convert_to_datetime(date_str):
    try:
        # First, handle the case where date_str is "0.0"
        if date_str == '0.0':
            return pd.NaT  # Return 'Not a Time'

        # Then try the standard date formats
        return pd.to_datetime(date_str, format='%Y-%m-%d %H:%M:%S')
    except ValueError:
        try:
            return pd.to_datetime(date_str, format='%Y-%m-%d') 
        except ValueError:
            return pd.NaT 

## Function - Set Final Path

In [76]:
def set_final_path(dataframe:pd.DataFrame) -> pd.DataFrame:
    """ 
    This function sets the final EBO path for cures, mods, PIFs, FC, SSDIL
    """
    # initiate lists to use
    list_cure = []
    list_mod = []
    list_pif = []
    list_fc = []
    list_ssdil = []
    
    # get the unique keys for reperf, liq, and pif paths
    list_cure = list(dataframe.loc[dataframe['Path_Reperf'].isin(['Cure'])]['PrimeKey_pbo'].values)
    list_mod = list(dataframe.loc[dataframe['Path_Reperf'].isin(['Mod'])]['PrimeKey_pbo'].values)
    list_pif = list(dataframe.loc[dataframe['path_pif'].isin(['PIF'])]['PrimeKey_pbo'].values)
    list_fc = list(dataframe.loc[dataframe['path_liq'].isin(['FC'])]['PrimeKey_pbo'].values)
    list_ssdil = list(dataframe.loc[dataframe['path_liq'].isin(['SSDIL'])]['PrimeKey_pbo'].values)

    # set the path_final columms
    # dataframe['path_final'] = 'PENDING'
    dataframe['path_final'] = np.where(dataframe.PrimeKey_pbo.isin(list_cure), 'Cure',
                                       np.where(dataframe.PrimeKey_pbo.isin(list_mod), 'Mod',
                                       np.where(dataframe.PrimeKey_pbo.isin(list_pif), 'PIF',
                                       np.where(dataframe.PrimeKey_pbo.isin(list_fc), 'FC',
                                       np.where(dataframe.PrimeKey_pbo.isin(list_ssdil), 'SSDIL', 'PENDING'))))
                                       )
    
    return dataframe

## Function - Set Final Path - Detailed

In [77]:
def set_final_path_detailed(dataframe:pd.DataFrame) -> pd.DataFrame:
    """ 
    This function sets the final EBO path for cures, mods, PIFs, FC, SSDIL
    """
    # initiate lists to use
    list_cure = []
    list_mod = []
    list_pif = []
    list_fc = []
    list_ssdil = []
    
    # get the unique keys for reperf, liq, and pif paths
    # list_cure = list(dataframe.loc[dataframe['Path_Reperf'].isin(['Cure'])]['PrimeKey_pbo'].values)
    # list_mod = list(dataframe.loc[dataframe['Path_Reperf'].isin(['Mod'])]['PrimeKey_pbo'].values)
    # list_pif = list(dataframe.loc[dataframe['path_pif'].isin(['PIF'])]['PrimeKey_pbo'].values)
    # list_fc = list(dataframe.loc[dataframe['path_liq'].isin(['FC'])]['PrimeKey_pbo'].values)
    # list_ssdil = list(dataframe.loc[dataframe['path_liq'].isin(['SSDIL'])]['PrimeKey_pbo'].values)
    set_pending_eligible = set(dataframe.loc[(dataframe['trade_period_min_pending_redeliverable_flag'] ==1)]['PrimeKey_pbo'].values)
    list_pending_eligible = list(set_pending_eligible)

    # set the path_final columms
    # dataframe['path_final_detailed'] = dataframe['path_final'] # might be able to skip this after using np.where
    dataframe['path_final_detailed'] = np.where(dataframe['path_final'] =='Cure', 'Cure',
                                       np.where(dataframe['path_final'] =='Mod', 'Mod',
                                       np.where(dataframe['path_final'] =='PIF', 'PIF',
                                       np.where(dataframe['path_final'] =='FC', 'FC',
                                       np.where(dataframe['path_final'] =='SSDIL', 'SSDIL',
                                       np.where(dataframe.PrimeKey_pbo.isin(list_pending_eligible), 'PENDING_REDELIVERABLE', 'PENDING_OTHER')))))
                                       )
    
    return dataframe

## Function - Get Max Trade Period

In [78]:
def get_max_trade_period(dataframe: pd.DataFrame)->pd.DataFrame:
    primekey_list = list(dataframe['PrimeKey_pbo'].unique())

    # initialize default values
    primekey_terminal_period_dict = {}
    dataframe['terminal_path_flag'] = 0

    for item in primekey_list:
        terminal_period = dataframe.loc[dataframe['PrimeKey_pbo']==item]['trade_period'].max()
        primekey_terminal_period_dict.update({item: terminal_period})
        # temp_df_prep_barings.loc[(temp_df_prep_barings['PrimeKey_pbo']==k) & (temp_df_prep_barings['PrimeKey_pbo']==terminal_period), 'trade_period']=1


    for i, (k, v) in enumerate(primekey_terminal_period_dict.items()):
        # temp_df_prep_barings.loc[(temp_df_prep_barings['PrimeKey_pbo']==k) & (temp_df_prep_barings['PrimeKey_pbo']==v), 'trade_period']=1
        # print(i, k, v) # used this during testing
        dataframe.loc[(dataframe['PrimeKey_pbo']==k) & (dataframe['trade_period']==v), 'terminal_path_flag']=1 # didn't work 
        # dataframe['terminal_path_flag'] = np.where(((dataframe['PrimeKey_pbo']==k) & (dataframe['trade_period']==v)),1,0)


    return dataframe
    

In [79]:
# df_test2 = data_test
# df_test2 = set_final_path(df_test2)

In [80]:
# df_test2['path_final'].head()

In [81]:
# quick_test = df_test2.loc[df_test2.PrimeKey_pbo.isin([10070485862105])]

In [82]:
# quick_test.PBOSale_SaleDate.values

In [83]:
# quick_test.PBOSale_SaleDate.dtype

In [84]:
# pd.to_datetime(dataframe['PBOSale_IntPaidToDate'])
# quick_test2 = quick_test

In [85]:
# quick_test2['test_saledate'] = pd.to_datetime(quick_test2['PBOSale_SaleDate'])

In [86]:
# new_df['PBOSale_SaleDate'].unique

In [87]:
# quick_test2['PBOSale_SaleDate'].values

In [88]:
# quick_test2 = prep_all_cashflows(quick_test)

In [89]:
# quick_test2.PBOSale_SaleDate.values

In [90]:
# print(new_df.loc[new_df['Path_Reperf'].isin(['Cure', 'Mod'])]['PrimeKey_pbo'].values)
# print(new_df.loc[new_df['Path_Reperf'].isin(['Cure', 'Mod'])][['PrimeKey_pbo', 'Path_Reperf']].values)
# print(new_df.loc[new_df.PrimeKey_pbo.isin([10002215992101, 10003567892101])][['PrimeKey_pbo', 'Path_Reperf', 'path_liq']].values)


In [91]:
# for values in unique PrimeKey_pbo
# df_barings_20240714['PrimeKey_pbo'].unique()

In [92]:
# for values in unique PrimeKey_pbo
# print(df_barings_20240714['PrimeKey_pbo'].unique())

In [93]:
# for values in unique PrimeKey_pbo
# temp_list = []
# temp_list = df_barings_20240714['PrimeKey_pbo'].unique().tolist # not working
# temp_list = list(df_barings_20240714['PrimeKey_pbo'].unique()) # this works

In [94]:
# len(temp_list)

## Function - Complete All Data Munging Steps and Cash Flows

In [124]:
# ADD FUNCTION TO DO ALL DATA MUNGING
# test to see if .function() will work
def prep_all_cashflows(dataframe: pd.DataFrame)->pd.DataFrame:    
    ## remove_extra_periods (function already created); remove_extra_periods(df_20200102_MM)
    # PENDING CHANGE THIS TO THE FILTER FUNCTION (IT'S QUICKER)
    dataframe = remove_extra_periods(dataframe) # TAKES TOO LONG
    print('Filtering out post redelivery periods')
    # dataframe = filter_out_extra_periods(dataframe) # <<<< FILTERING IN SQL DUE TO FILTERING GLITCHES
    
    ## Convert PBOSale_IntPaidToDate to datetime
    dataframe['PBOSale_IntPaidToDate'] = pd.to_datetime(dataframe['PBOSale_IntPaidToDate'])


    ## Convert PBOSale_SaleDate to datetime
    # dataframe['PBOSale_SaleDate'] = dataframe['PBOSale_SaleDate'].apply(lambda x: pd.to_datetime(str(x),format='%Y%m%d'))
    dataframe['PBOSale_SaleDate'] = pd.to_datetime(dataframe['PBOSale_SaleDate'])
    # VVVV OLD SOLUTION
    # dataframe['PBOSale_SaleDate'] = dataframe['PBOSale_SaleDate'].apply(
    #     lambda x: pd.to_datetime(str(x).split()[0], format='%Y-%m-%d') # Split on space, take first part
    #     ) # gemini version
    # ^^^^ OLD SOLUTION
    # VVVV revert back to this version that worked in the past?
    # df_test_trimmed['PBOSale_SaleDate'] = df_test_trimmed['PBOSale_SaleDate'].apply(lambda x: pd.to_datetime(str(x),format='%Y%m%d'))
    # dataframe['PBOSale_SaleDate'] = dataframe['PBOSale_SaleDate'].apply(convert_to_datetime) 
    # ^^^^ convert to datetime function might be causing errors

    ## Calculate Days360 (function already created)
    print('Calculating DAYS360 for accrued interest at settlement')
    dataframe['DAYS360_DAYS'] = dataframe.apply(
        lambda row: days360Calc(row['PBOSale_IntPaidToDate'], row['PBOSale_SaleDate']),
        axis=1)
    
    ## Calculate Accrued Interest
    dataframe['ACCRUED_INTEREST'] = round((dataframe['DAYS360_DAYS'] 
                                           * dataframe['PBOSale_SaleUPB'] 
                                           * dataframe['PBOSale_SalePassThru'] 
                                           / 360),2)


    ## Calculate Investor Principal - Funding Schedule at Settlement
    dataframe['orig_settle_fs_investor_principal'] = round((dataframe['PBOSale_Price'] 
                                                     * dataframe['PBOSale_SaleUPB']),2)

    ## Calculate Investor Interest - Funding Schedule at Settlement
    dataframe['orig_settle_fs_investor_interest'] = round(dataframe['ACCRUED_INTEREST'],2)

    ## Calculate Pennymac Funding Schedule at Settlement
    # vvvv OLD orig_settle_fs_penny LOGIC
    # dataframe['orig_settle_fs_penny'] = round(((dataframe['PBOSale_Price'] - 1.0) 
    #                                            * dataframe['PBOSale_SaleUPB']),2)
    # ^^^^ OLD orig_settle_fs_penny LOGIC
    # vvvv NEW LOGIC
    dataframe['orig_settle_fs_penny_principal'] = round((dataframe['PBOSale_Price'] 
                                                     * dataframe['PBOSale_SaleUPB']),2)

    dataframe['orig_settle_fs_penny_interest'] = round(dataframe['ACCRUED_INTEREST'],2)
    # ^^^^ NEW LOGIC

    # Create Pennymac cash flows with signs
    dataframe['penny_fs_principal_paidto_gnma'] = -dataframe['PBOSale_SaleUPB'] # paid to GNMA at settlement for buyout
    dataframe['penny_fs_interest_paidto_gnma'] = -round(dataframe['ACCRUED_INTEREST'],2)
    dataframe['penny_principal_paidto_investor'] = -dataframe['ia_total_SSEPrincipal']
    dataframe['penny_netinterest_paidto_investor'] = -dataframe['ia_total_SSENetinterest']




    print('Funding Schedule calcs completed')

    ## Calculate Redelivery Fees
    print('Setting redelivery fees and repurchase interest')
    dataframe['redelivery_fee'] = 0
    dataframe.loc[(dataframe['Buy Price'] > 100) & (dataframe['Shared Econ'] > 0), ['redelivery_fee']] = 300
    dataframe.loc[(dataframe['Buy Price'] < 100) & (dataframe['Buy Price'] > 0) & (dataframe['EBO_Investor_y'] != 'Goldman'), ['redelivery_fee']] = 300
    dataframe.loc[(dataframe['Buy Price'] > 100) & (dataframe['EBO_Investor_y'] == 'Goldman'), ['redelivery_fee']] = 150

    ## Calculate Repurchase Par Amount
    dataframe['investor_repurch_par_dlr'] = np.where((dataframe['Face Amount'])>0, dataframe['Face Amount'] * 1000, 0)
    dataframe['penny_repurchase_par_amount'] = -dataframe['investor_repurch_par_dlr']
    dataframe['penny_repurchase_px_x_repurch_amount'] = -dataframe['investor_repurch_par_dlr'] * dataframe['Buy Price'] / 100 # add penny repurchase px x upb

    ## NEED TO ADD REPURCHASE INTEREST
    # set InterestPaidToDt to datetime
    dataframe['InterestPaidToDt'] = pd.to_datetime(dataframe['InterestPaidToDt'])

    # set repurchase_date_calc to datetime
    dataframe['repurchase_date_calc'] = pd.to_datetime(dataframe.loc[dataframe['repurchase_date_calc']!='0']['repurchase_date_calc'])

    # calculate interest days
    dataframe['repurchase_interest_days'] = np.where(dataframe['Management'] =='EBO', 
                                              dataframe.apply(
                                                  lambda row: days360Calc(row['InterestPaidToDt'], row['repurchase_date_calc']),
                                                  axis=1),
                                                  0)

    # calculate repurchase interest 
    dataframe['repurchase_interest'] = round((dataframe['repurchase_interest_days'] 
                                              * dataframe['Face Amount'] * 1000
                                              * (dataframe['CurrentInterestRate'] - 0.0025) 
                                              / 360),2)
    
    dataframe['penny_repurchase_interest_paidto_investor'] = -dataframe['repurchase_interest'] # show penny payment of interest to investor

    # add placeholder for penny redelivery into MBS; will likely apply once redirect script to more stable qrm table
    dataframe['penny_redelivery_px_x_repurch_upb'] = -dataframe['penny_repurchase_px_x_repurch_amount'] # penny redelivery into MBS; placeholder until redirect script


    # calculate 

    # ## FC SSDIL Remittance Liquidation Calculations 


    ## Calculate Pennymac PT Savings
    # dataframe['penny_pt_savings'] = round(dataframe['CurrentMonthlyPaymentAmt'],2) # added into script
    print('Calculating Penny PT savings')
    # need to add filter to set to zero when loan reperforming
    dataframe['penny_pt_savings_orig'] = round(dataframe['penny_pt_savings_orig'],2) # added to SQL script (PandI - Servicing Amount)

    # net out any principal and interest payments paid to investor
    dataframe['penny_pt_savings'] = (dataframe['penny_pt_savings_orig']
                                     - dataframe['ia_total_SSEPrincipal']
                                     - dataframe['ia_total_SSENetinterest']
                                     )
    
    # OLD SOLUTION BEFORE FILTER
    # vvvv dont zero out pt savings when current. Using different methodology for this portion
    # dataframe['penny_pt_savings'] = np.where((dataframe['LoanStatusId'].isin(['P'])), # FILTER FOR PERFORMING LOANS
    #                                          0,
    #                                          round(dataframe['penny_pt_savings'],2))
    # ^^^^ dont zero out pt savings when current. Using different methodology for this portion


    # set the path_final columms
    print('Setting final path')
    dataframe = set_final_path(dataframe)

    # Change sign of original settle numbers for charts downstream
    print('Changing sign for Funding Schedule principal and interest')
    dataframe['orig_settle_fs_investor_interest'] = dataframe['orig_settle_fs_investor_interest'] * -1
    dataframe['orig_settle_fs_investor_principal'] = dataframe['orig_settle_fs_investor_principal'] * -1

    # Get terminal path trade_period for each path
    print('Getting max trade period')
    dataframe = get_max_trade_period(dataframe)

    # Get Original Trade UPB at Terminal UPB # 
    dataframe['terminal_path_orig_trade_upb'] = (dataframe['terminal_path_flag'] *
                                                 dataframe['PBOSale_SaleUPB_Orig'])
    
    # Create terminal flag x UPB
    dataframe['terminal_path_x_current_upb'] = (dataframe['terminal_path_flag'] *
                                                 dataframe['CurrentPrincipalBalanceAmt'])
    
    # Create terminal flag x cumulative_principal_collected_ia
    dataframe['terminal_path_flag_x_cum_prin_ia'] = (dataframe['terminal_path_flag'] *
                                                 dataframe['cumulative_principal_collected_ia']) 

    # cumulative_interest_collected_ia  cumulative_netinterest_collected_ia cumulative_sfee_collected_ia                                            
    dataframe['terminal_path_flag_x_cum_interest_ia'] = (dataframe['terminal_path_flag'] *
                                                 dataframe['cumulative_interest_collected_ia']) 
    dataframe['terminal_path_flag_x_cum_net_interest_ia'] = (dataframe['terminal_path_flag'] *
                                                 dataframe['cumulative_netinterest_collected_ia']) 
    dataframe['terminal_path_flag_x_cum_sfee_ia'] = (dataframe['terminal_path_flag'] *
                                                 dataframe['cumulative_sfee_collected_ia']) 

    # Create terminal flag x DQ and x (DQ x terminal UPB)
    dataframe['terminal_path_flag_x_dq_months'] = (dataframe['terminal_path_flag'] *
                                                 dataframe['dq_months']) 
    dataframe['terminal_path_flag_x_dq_months_x_term_upb'] = (dataframe['terminal_path_x_current_upb'] *
                                                 dataframe['terminal_path_flag_x_dq_months']) 
    
    # Create terminal flag x PT and x (PT x terminal UPB)
    dataframe['terminal_path_flag_x_mbs_pt_rate'] = (dataframe['terminal_path_flag'] *
                                                 dataframe['mbs_pt_rate']) 
    dataframe['terminal_path_flag_x_mbs_pt_rate_x_term_upb'] = (dataframe['terminal_path_x_current_upb'] *
                                                 dataframe['terminal_path_flag_x_mbs_pt_rate']) 
    
    # Calculate total principal collected + redelivery par amount
    dataframe['terminal_prin_collected_plus_redelivered_amount'] = (dataframe['terminal_path_flag_x_cum_prin_ia'] + 
                                                                    dataframe['investor_repurch_par_dlr'])
    
    # add clean_current_payments and loanstatus at terminal period (for pending loan exhibit)
    dataframe['terminal_pending_cleanpay'] = np.where(((dataframe['terminal_path_flag']==1) & (dataframe['path_final']=='PENDING')),
                                                       dataframe['clean_current_payments'], 0)

    dataframe['terminal_pending_loanstatus'] = np.where(((dataframe['terminal_path_flag']==1) & (dataframe['path_final']=='PENDING')),
                                                       dataframe['LoanStatusId'], 0)

    # create the wire date to use based on normal remittance date vs repurchase date
    # first we need to set PBOSale_SaleDate to date dtype from object; might have to do fillna first
    # dataframe['PBOSale_SaleDate'] = dataframe['PBOSale_SaleDate'].fillna(0) 
    dataframe['PBOSale_SaleDate'] = pd.to_datetime(dataframe['PBOSale_SaleDate'])
    dataframe['PBOSale_SaleDate'] = dataframe['PBOSale_SaleDate'].dt.strftime('%Y-%m-%d') # added this
    # might have to do the same for the repurchase_date_calc and wire_date_remit
    dataframe['repurchase_date_calc'] = pd.to_datetime(dataframe['repurchase_date_calc'])
    dataframe['repurchase_date_calc'] = dataframe['repurchase_date_calc'].dt.strftime('%Y-%m-%d') 
    dataframe['wire_date_remit'] = pd.to_datetime(dataframe['wire_date_remit'])
    dataframe['wire_date_remit'] = dataframe['wire_date_remit'].dt.strftime('%Y-%m-%d') 


    dataframe['wire_date_final'] = np.where(dataframe['trade_period'] == 0, dataframe['PBOSale_SaleDate'],
                                            np.where((dataframe['path_final'].isin(['Cure','Mod']) & dataframe['terminal_path_flag'] == 1), 
                                                    dataframe['repurchase_date_calc'], dataframe['wire_date_remit'])
                                                    )


    # Create path_final_detail column for path_final == PENDING loans 
    # and set redelivery eligible path flag
    dataframe = get_min_pending_redelivery_eligible_period(dataframe)


    # use np.where to set the path_final_detail column
    # find path_final function sent new flag when not PENDING, if PENDING 1 then PENDING REDELIVERABLE, ELSE PENDING OTHER
    dataframe = set_final_path_detailed(dataframe)


    # Set terminial detailed flag. This flag accounts for PENDING_REDELIVERABLE loans
    dataframe['terminal_path_flag_detailed'] = np.where(dataframe['path_final_detailed'] =='PENDING_REDELIVERABLE', 
                                                        dataframe['trade_period_min_pending_redeliverable_flag'], 
                                                        dataframe['terminal_path_flag'])


    # multiply terminal_path_flag_detailed x tradeupb and upb
    # Create terminal flag x DQ and x (DQ x terminal UPB)
    dataframe['terminal_path_flag_detailed_x_tradeupb'] = (dataframe['terminal_path_flag_detailed'] *
                                                 dataframe['PBOSale_SaleUPB_Orig']) 
    dataframe['terminal_path_flag_detailed_x_upb'] = (dataframe['terminal_path_flag_detailed'] *
                                                 dataframe['CurrentPrincipalBalanceAmt']) 
    

    # For WAL, multiply terminal period (orig and detailed) times upb, terminal_path_x_current_upb, terminal_path_x_current_upb
    dataframe['terminal_path_x_current_upb_x_trade_period'] = (dataframe['terminal_path_x_current_upb'] *
                                                 dataframe['trade_period']) 
    dataframe['terminal_path_flag_detailed_x_upb_x_trade_period'] = (dataframe['terminal_path_flag_detailed_x_upb'] *
                                                 dataframe['trade_period']) 



    # vvvv MOVED REMIT STUFF HERE vvvv
    ## FC SSDIL Remittance Liquidation Calculations 
    print('Calculating FC remittance amounts')

    dataframe['CurrentPrincipalBalanceAmt_next'] = pd.to_numeric(dataframe['CurrentPrincipalBalanceAmt_next'], errors='coerce')

    # set remit dates to datetime
    dataframe['remit_default_dt'] = pd.to_datetime(dataframe.loc[dataframe['remit_default_dt'].notnull()]['remit_default_dt'], errors='coerce')
    dataframe['remit_MinTransactiondate'] = pd.to_datetime(dataframe.loc[dataframe['remit_MinTransactiondate'].notnull()]['remit_MinTransactiondate'], errors='coerce')
    dataframe['remit_MaxTransactiondate'] = pd.to_datetime(dataframe.loc[dataframe['remit_MaxTransactiondate'].notnull()]['remit_MaxTransactiondate'], errors='coerce')
    dataframe['remit_eom_TransactionDate'] = pd.to_datetime(dataframe.loc[dataframe['remit_eom_TransactionDate'].notnull()]['remit_eom_TransactionDate'], errors='coerce')

    # handle NaN values for remit_days_split_interest and remit_days_default_dt otherwise calcs will yield NaN
    # dataframe[['remit_days_split_interest', 'remit_days_default_dt']] = dataframe[['remit_days_split_interest', 'remit_days_default_dt']].fillna(0) # did not work when list. try individually
    dataframe['remit_days_split_interest'] = dataframe['remit_days_split_interest'].fillna(0)
    dataframe['remit_days_default_dt'] = dataframe['remit_days_default_dt'].fillna(0) 
    dataframe['FHA_DebentureRt'] = dataframe['FHA_DebentureRt'].fillna(0) 
    dataframe['ia_cumulative_total_interest_fc'] = dataframe['ia_cumulative_total_interest_fc'].fillna(0) 
    # might have to do this for FHA_DebentureRt

    # vvvv OLD SOLUTION vvvv
    # dataframe['remit_expected_claims_interest'] = np.where(((dataframe['LoanType'] =='FHA') & 
    # dataframe['remit_expected_claims_interest'] = np.where(((dataframe['LoanType'].isin(['FHA'])) & 
    #                                                         (dataframe['terminal_path_flag']==1) &
    #                                                         (dataframe['LoanSubStatusId'].isin(['LSS', 'LDL', '3F', 'LFC', 'LU', 'LUS', 'LRS'])) & 
    #                                                         (dataframe['CurrentPrincipalBalanceAmt_next'] <1)), 
    #                                                        (dataframe['remit_upb_preliquidation'] * (dataframe['FHA_DebentureRt'] / 100) * dataframe['remit_days_split_interest'] / 365) + (dataframe['remit_expected_claims_upb'] * (dataframe['FHA_DebentureRt'] / 100) * dataframe['remit_days_default_dt'] / 365) - dataframe['ia_cumulative_total_interest_fc'],
    #                                                        np.where(((dataframe['LoanType'].isin(['VA', 'USDA'])) & 
    #                                                                  (dataframe['terminal_path_flag']==1) &
    #                                                                  (dataframe['LoanSubStatusId'].isin(['LSS', 'LDL', '3F', 'LFC', 'LU', 'LUS', 'LRS'])) & 
    #                                                                  (dataframe['CurrentPrincipalBalanceAmt_next'] <1)),
    #                                                                 ((dataframe['remit_upb_preliquidation'] * (dataframe['CurrentInterestRate'] - 0.0025) * dataframe['remit_days_split_interest'] / 365) 
    #                                                                 + (dataframe['remit_days_split_interest'] * (dataframe['CurrentInterestRate'] - 0.0025) * dataframe['remit_days_default_dt'] / 365) 
    #                                                                 - dataframe['ia_cumulative_total_interest_fc']), 
    #                                                                 0)
    #                                                                 )
    # ^^^^ OLD SOLUTION ^^^^
    # vvvv NEW SOLUTION vvvv
    dataframe['remit_expected_claims_interest'] = np.where((dataframe['LoanType'].isin(['FHA'])),
                                                 ((dataframe['remit_upb_preliquidation'] * (dataframe['FHA_DebentureRt'] / 100) * dataframe['remit_days_split_interest'] / 365) + 
                                                  (dataframe['remit_expected_claims_upb'] * (dataframe['FHA_DebentureRt'] / 100) * dataframe['remit_days_default_dt'] / 365) - 
                                                  dataframe['ia_cumulative_total_interest_fc']),
                                                  ((dataframe['remit_upb_preliquidation'] * (dataframe['CurrentInterestRate'] - 0.0025) * dataframe['remit_days_split_interest'] / 365) + 
                                                   (dataframe['remit_expected_claims_upb'] * (dataframe['CurrentInterestRate'] - 0.0025) * dataframe['remit_days_default_dt'] / 365) - 
                                                   dataframe['ia_cumulative_total_interest_fc'])) # this works
    
    # clean up FC non-terminal periods
    dataframe['remit_expected_claims_interest'] = dataframe['remit_expected_claims_interest'] * dataframe['terminal_path_flag']
    dataframe['remit_expected_claims_upb'] = dataframe['remit_expected_claims_upb'] * dataframe['terminal_path_flag']
    dataframe['remit_upb_preliquidation'] = dataframe['remit_upb_preliquidation'] * dataframe['terminal_path_flag']
    dataframe['remit_sales_proceeds'] = dataframe['remit_sales_proceeds'] * dataframe['terminal_path_flag']
    dataframe['remit_days_split_interest'] = dataframe['remit_days_split_interest'] * dataframe['terminal_path_flag']
    dataframe['remit_days_default_dt'] = dataframe['remit_days_default_dt'] * dataframe['terminal_path_flag']

    # add instance of interest paid in prior periods for use in FC remittance tie-out file
    dataframe['remit_interest_paid_prior_periods'] = dataframe['ia_cumulative_total_interest_fc'] * dataframe['terminal_path_flag'] # ia_cumulative_total_interest_fc
    # ^^^^ NEW SOLUTION ^^^^

    dataframe['remit_total_due_investor'] = np.where(((dataframe['LoanSubStatusId'].isin(['LSS', 'LDL', '3F','FS', 'LFC', 'LU', 'LUS', 'LRS'])) & 
                                                      (dataframe['terminal_path_flag']==1) &
                                                      (dataframe['CurrentPrincipalBalanceAmt_next'] <1)), 
                                                      (dataframe['remit_expected_claims_upb'] + dataframe['remit_expected_claims_interest']), 
                                                      0)
    
    # ADD REMOVAL OF FINAL PERIOD ia_total_sseprincipal and ia_total_sseNetinterest...also check fc version of those columns
    # ^^^^ MOVED REMIT STUFF HERE ^^^^

    # Check to see if total prin equals original trade upb
    # add np.where or set column to zero then calc for non-pending loans
    dataframe['terminal_total_prin_minus_origtradeupb'] = 0 # initialize the column. Prob
    dataframe['terminal_total_prin_minus_origtradeupb'] = np.where(((dataframe['path_final'].isin(['Cure', 'Mod', 'PIF', 'FC', 'SSDIL'])) & (dataframe['terminal_path_flag']==1)),
                                             (dataframe['terminal_prin_collected_plus_redelivered_amount'] - dataframe['terminal_path_orig_trade_upb']),0)
    
    # Allocate prin shortfalls to Pennymac for Cures and Mods initiate with zero values first
    dataframe['penny_prin_from_shortfall'] = 0
    dataframe['penny_prin_from_shortfall'] = np.where(((dataframe['path_final'].isin(['Cure', 'Mod'])) & 
                                                       (dataframe['terminal_path_flag']==1) & 
                                                       (dataframe['terminal_total_prin_minus_origtradeupb'] < 0)),
                                             (dataframe['terminal_total_prin_minus_origtradeupb'] * (-1)),0) # need to change sign from negative to positive

    
    # Calculate Penny Debenture Rate Savings
    print('Calculating Penny debrate savings')

    # prep fields for calcs
    # dataframe['FHA_DebentureRt'] = dataframe['FHA_DebentureRt'].fillna(0) 

    dataframe['penny_debrate_savings'] = np.where((dataframe['path_final'].isin(['FC', 'SSDIL'])) & (dataframe['terminal_path_flag']==1) & (dataframe['LoanType']=='FHA'), # FILTER FOR FHA
                                            #  ((dataframe['CurrentInterestRate'] - 0.0025 - dataframe['FHA_DebentureRt']/100) * # PENDING - NEED TO UPDATE THIS WITH THE MBS PASS-THROUGH
                                             ((dataframe['mbs_pt_rate'] - dataframe['FHA_DebentureRt'] / 100) * # <<< DONE - NEED TO UPDATE THIS WITH THE MBS PASS-THROUGH
                                             ((dataframe['remit_days_split_interest'] + dataframe['remit_days_default_dt'] - 60) / 365) *
                                             (dataframe['remit_upb_preliquidation'] + dataframe['remit_expected_claims_upb'])
                                             ),0)
    
    # fill missing values
    print('Filling na values')
    dataframe = dataframe.fillna(0) # this fixes the null total column issue

    # remove duplicates # had missing loans from trade TO 202203
    print('\nRemoving duplicate rows')
    rows_before_removal = dataframe.shape[0]
    print(f'\tRows before removal: {rows_before_removal}')
    dataframe = dataframe.drop_duplicates()
    rows_after_removal = dataframe.shape[0]
    print(f'\tRows after removal: {rows_after_removal}')
    print(f'\tRows removed: {rows_before_removal - rows_after_removal}\n')

    ## Calculate Investor Total Cashflows
    print('Calculating total cashflows')
    dataframe['total_cashflows_investor'] = (dataframe['orig_settle_fs_investor_interest']
                                             + dataframe['orig_settle_fs_investor_principal']
                                             + dataframe['investor_redelivery_premium_dlr']
                                             + dataframe['repurchase_interest']
                                             + dataframe['ia_total_SSEPrincipal'] 
                                             + dataframe['ia_total_SSENetinterest'] 
                                             - dataframe['redelivery_fee']
                                             + dataframe['investor_repurch_par_dlr']
                                             + dataframe['remit_total_due_investor']
                                             ) 

    # adjust for double-counting prin and net interest in last period by removing prin and net int 
    dataframe['total_cashflows_investor'] = np.where(((dataframe['LoanSubStatusId'].isin(['LSS', 'LDL', '3F','FS', 'LFC', 'LU', 'LUS', 'LRS'])) & 
                                                      (dataframe['terminal_path_flag']==1) &
                                                      (dataframe['CurrentPrincipalBalanceAmt_next'] <1)), 
                                                      (dataframe['total_cashflows_investor'] 
                                                       - dataframe['ia_total_SSEPrincipal'] 
                                                       - dataframe['ia_total_SSENetinterest']
                                                       ), 
                                                      dataframe['total_cashflows_investor'])


    ## Calculate Pennymac Total Cashflows
    dataframe['total_cashflows_penny'] = (dataframe['orig_settle_fs_penny_principal'] # replaced this term dataframe['orig_settle_fs_penny']
                                          + dataframe['orig_settle_fs_penny_interest']
                                          + dataframe['penny_fs_principal_paidto_gnma'] # principal paid to GNMA for buyout # negative number
                                          + dataframe['penny_fs_interest_paidto_gnma'] # interest  paid to GNMA for buyout # negative number
                                        #   + dataframe['penny_principal_paidto_investor'] # regular borrower principal payments # negative number # removed from cash flow, left for reference
                                        #   + dataframe['penny_netinterest_paidto_investor'] # regular borrower interest payments net of sfee # negative number # removed from cash flow, left for reference
                                          + dataframe['ia_total_SSEServiceFee']
                                          # + (dataframe['penny_repurchase_par_amount'] * dataframe['Buy Price'] / 100) # <<<< PENDING double-check to see if shared econ affects buy price. I think it does # face amount paid to investor at repurchase # negative number
                                          + dataframe['redelivery_fee']
                                          + dataframe['penny_repurchase_px_x_repurch_amount'] # add penny repurchase px x upb pays to investor
                                          + dataframe['penny_repurchase_interest_paidto_investor'] # per diem repurchase interest penny pays to investor; negative number
                                          + dataframe['penny_redelivery_px_x_repurch_upb'] # penny redelivery into MBS; placeholder until redirect script
                                          + dataframe['penny_redelivery_premium_dlr']
                                          # - dataframe['investor_repurch_par_dlr'] # face amount paid to investor at repurchase
                                        #   + dataframe['penny_pt_savings_orig'] # remove this since not an actual cash flow. Can use it to evaluate NPV for leaving in pool.
                                          # <<<< PENDING add dataframe['penny_debrate_savings']
                                          + dataframe['penny_debrate_savings']
                                          + dataframe['penny_prin_from_shortfall'] # principal shortfall from late-month redelivery activity
                                          ) # might have to add in liquidation cashflows depending on findings
    
    return dataframe

## TESTING

In [96]:
# test1 = new_df.loc[(new_df['LoanId'].isin([8001288834]))]

In [97]:
# list of fc loans for test # '1000154904'	,'1000170928'	, '1000194906'
# test_list = [1003769865, 1004135957, 1005054082, 1005125950, 8000020066, 8004613981, 8012329905, 8013181209, 8015663261, 8016776771, 8024023149, 8026897006, 8029400491, 8031198342, 1000949937, 8015920203, 8016866814, 8023525068, 8031723910]
test_list = [1000154904, 1000170928, 1000194906]

In [98]:
print(test_list)

[1000154904, 1000170928, 1000194906]


In [99]:
# test1 = new_df.loc[(new_df['LoanId'].isin(test_list))]
# test_final = data_final.loc[(data_final['LoanId'].isin(test_list))]

In [100]:
# test1.head()
# test_final.head()

In [101]:
# test_final = get_min_pending_redelivery_eligible_period(test_final)

In [102]:
# test wire_date_final
# test_final['wire_date_final'] = np.where(test_final['trade_period'] == 0, test_final['PBOSale_SaleDate'],
#                                          np.where((test_final['path_final'].isin(['Cure','Mod']) & test_final['terminal_path_flag'] == 1), 
#                                                   test_final['repurchase_date_calc'], test_final['wire_date_remit'])
#                                                   )

In [103]:
# test_final.to_csv(f'C:\\Users\\{login}\\Desktop\\test_final_{date_string}_202203SaleMonth.csv', index=True, index_label='Index')

In [104]:
# print(set(test_final.loc[(test_final['trade_period_min_pending_redeliverable_flag'] ==1)]['PrimeKey_pbo'].values)) # {10001549042203}
# print(list(test_final.loc[(test_final['trade_period_min_pending_redeliverable_flag'] ==1)]['PrimeKey_pbo'].values)) # [10001549042203]

In [105]:
# test_final2 = set_final_path_detailed(test_final)

In [106]:
# test_final2.to_csv(f'C:\\Users\\{login}\\Desktop\\test_final2_{date_string}_202203SaleMonth.csv', index=True, index_label='Index')

In [107]:
# test1['remit_expected_claims_interest'] = 0
# test1['remit_expected_claims_interest'] = test1.loc[((test1['LoanType'].isin(['FHA'])) & 
#                                                      (test1['terminal_path_flag']==1) &
#                                                      (test1['LoanSubStatusId'].isin(['LSS', 'LDL', '3F', 'LFC', 'LU', 'LUS', 'LRS'])) & 
#                                                      (test1['CurrentPrincipalBalanceAmt_next'] <1)), 'remit_expected_claims_interest'] = ((test1['remit_upb_preliquidation'] * (test1['FHA_DebentureRt'] / 100) * test1['remit_days_split_interest'] / 365) + 
#                                                                                                                                           (test1['remit_expected_claims_upb'] * (test1['FHA_DebentureRt'] / 100) * test1['remit_days_default_dt'] / 365) - 
#                                                                                                                                           test1['ia_cumulative_total_interest_fc'])
                                                        # np.where(((test1['LoanType'].isin(['VA', 'USDA'])) & 
                                                        #             (test1['terminal_path_flag']==1) &
                                                        #             (test1['LoanSubStatusId'].isin(['LSS', 'LDL', '3F', 'LFC', 'LU', 'LUS', 'LRS'])) & 
                                                        #             (test1['CurrentPrincipalBalanceAmt_next'] <1)),
                                                        #         (test1['remit_upb_preliquidation'] * (test1['CurrentInterestRate'] - 0.0025) * test1['remit_days_split_interest'] / 365) 
                                                        #         + (test1['remit_days_split_interest'] * (test1['CurrentInterestRate'] - 0.0025) * test1['remit_days_default_dt'] / 365) 
                                                        #         - test1['ia_cumulative_total_interest_fc'], 
                                                        #         0)
                                                        #         )


In [108]:
# test1.loc[test1['terminal_path_flag']==1]['remit_expected_claims_interest'].values
# test1.loc[test1['terminal_path_flag']==1][['remit_default_dt', 'remit_MinTransactiondate', 'remit_MaxTransactiondate', 'remit_eom_TransactionDate','remit_expected_claims_interest']].values

# vvvv this showed NaN for remit_days_split_interest and remit_days_default_dt
# test1 is new_df not data_final though
# test_final.loc[test_final['terminal_path_flag']==1][(['remit_default_dt', 'remit_MinTransactiondate', 
#                                             'remit_MaxTransactiondate', 'remit_eom_TransactionDate',
#                                             'remit_expected_claims_interest', 'remit_days_split_interest', 
#                                             'remit_days_default_dt', 'remit_total_due_investor', 'ia_cumulative_total_interest_fc'])].head(19)

In [109]:
# test split interest calcs - this works for remit_upb_preliquidation and CurrentInterestRate and remit_days_default_dt
# test_final['test_split_interest'] = test_final['remit_days_default_dt'] * test_final['CurrentInterestRate'] / 365 * test_final['remit_upb_preliquidation'] # works

# add case for fha vs vs/usda
# test_final['test_split_interest'] = np.where((test_final['LoanType'].isin(['FHA'])),
#                                              ((test_final['remit_upb_preliquidation'] * (test_final['FHA_DebentureRt'] / 100) * test_final['remit_days_split_interest'] / 365) + 
#                                               (test_final['remit_expected_claims_upb'] * (test_final['FHA_DebentureRt'] / 100) * test_final['remit_days_default_dt'] / 365) - 
#                                               test_final['ia_cumulative_total_interest_fc']),
#                                               ((test_final['remit_upb_preliquidation'] * (test_final['CurrentInterestRate'] - 0.0025) * test_final['remit_days_split_interest'] / 365) + 
#                                                (test_final['remit_days_split_interest'] * (test_final['CurrentInterestRate'] - 0.0025) * test_final['remit_days_default_dt'] / 365) - 
#                                                test_final['ia_cumulative_total_interest_fc'])) # this works

In [110]:
# just terminal period
# test_final.loc[test_final['terminal_path_flag']==1][(['PrimeKey_pbo', 'LoanType', 'remit_expected_claims_interest', 'remit_days_split_interest', 
#                                             'remit_days_default_dt', 'remit_total_due_investor', 'test_split_interest'])].head(19)

# all periods
# test_final[(['PrimeKey_pbo', 'LoanType', 'trade_period', 'remit_expected_claims_interest', 'remit_days_split_interest', 
#              'remit_days_default_dt', 'remit_total_due_investor', 'test_split_interest'])].head(40)

In [111]:
# test1_final = prep_all_cashflows(new_df)

In [112]:
# test1_final.shape

In [113]:
# test_final.to_csv(f'C:\\Users\\{login}\\Desktop\\test_final_{date_string}_202203SaleMonth.csv', index=True, index_label='Index')

In [114]:
# test1 = new_df.copy()

In [115]:
# print(test1[['remit_default_dt', 'remit_MinTransactiondate', 'remit_MaxTransactiondate', 'remit_eom_TransactionDate']].dtypes)
# print(test1[['remit_upb_preliquidation', 'FHA_DebentureRt', 'ia_cumulative_total_interest_fc']].dtypes)
# print(test1.loc[test1['remit_default_dt']!=NaT]['remit_default_dt', 'remit_MinTransactiondate', 'remit_MaxTransactiondate', 'remit_eom_TransactionDate'].head())
# print(new_df.loc[new_df['remit_default_dt'].notnull() ]['remit_default_dt'].head()) # this appears to work
# test1[['remit_default_dt', 'remit_MinTransactiondate', 'remit_MaxTransactiondate', 'remit_eom_TransactionDate']] = pd.to_datetime(test1[test1['remit_default_dt'].notnull()]['remit_default_dt'][['remit_default_dt', 'remit_MinTransactiondate', 'remit_MaxTransactiondate', 'remit_eom_TransactionDate']], errors='coerce')
# test1['remit_default_dt'] = pd.to_datetime(test1.loc[test1['remit_default_dt'].notnull()]['remit_default_dt'], errors='coerce')
# test1['remit_default_dt'] = pd.to_datetime(test1.loc[test1['remit_default_dt'].notnull()]['remit_default_dt'], errors='coerce').strftime('%Y-%m-%d') # error
# test1['remit_default_dt'] = pd.to_datetime(test1.loc[test1['remit_default_dt'].notnull()]['remit_default_dt'], errors='coerce').date()


In [116]:
# print(new_df.loc[new_df['remit_default_dt'].notnull() ]['remit_default_dt'].head()) # this appears to work
# test1[['remit_default_dt', 'remit_MinTransactiondate', 'remit_MaxTransactiondate', 'remit_eom_TransactionDate']] = pd.to_datetime(test1[test1['remit_default_dt'].notnull()]['remit_default_dt'][['remit_default_dt', 'remit_MinTransactiondate', 'remit_MaxTransactiondate', 'remit_eom_TransactionDate']], errors='coerce')
# test1['remit_MinTransactiondate'] = pd.to_datetime(test1.loc[test1['remit_MinTransactiondate'].notnull()]['remit_MinTransactiondate'], errors='coerce')


In [117]:
# print(new_df.loc[new_df['remit_default_dt'].notnull() ]['remit_default_dt'].head()) # this appears to work
# test1[['remit_default_dt', 'remit_MinTransactiondate', 'remit_MaxTransactiondate', 'remit_eom_TransactionDate']] = pd.to_datetime(test1[test1['remit_default_dt'].notnull()]['remit_default_dt'][['remit_default_dt', 'remit_MinTransactiondate', 'remit_MaxTransactiondate', 'remit_eom_TransactionDate']], errors='coerce')
# test1['remit_MaxTransactiondate'] = pd.to_datetime(test1.loc[test1['remit_MaxTransactiondate'].notnull()]['remit_MaxTransactiondate'], errors='coerce')

In [118]:
# print(new_df.loc[new_df['remit_default_dt'].notnull() ]['remit_default_dt'].head()) # this appears to work
# test1[['remit_default_dt', 'remit_MinTransactiondate', 'remit_MaxTransactiondate', 'remit_eom_TransactionDate']] = pd.to_datetime(test1[test1['remit_default_dt'].notnull()]['remit_default_dt'][['remit_default_dt', 'remit_MinTransactiondate', 'remit_MaxTransactiondate', 'remit_eom_TransactionDate']], errors='coerce')
# test1['remit_eom_TransactionDate'] = pd.to_datetime(test1.loc[test1['remit_eom_TransactionDate'].notnull()]['remit_eom_TransactionDate'], errors='coerce')

In [125]:
data_final = prep_all_cashflows(new_df)

Filtering out post redelivery periods
Calculating DAYS360 for accrued interest at settlement
Funding Schedule calcs completed
Setting redelivery fees and repurchase interest
Calculating Penny PT savings
Setting final path
Changing sign for Funding Schedule principal and interest
Getting max trade period
Calculating FC remittance amounts
Calculating Penny debrate savings
Filling na values

Removing duplicate rows
	Rows before removal: 13845
	Rows after removal: 13842
	Rows removed: 3

Calculating total cashflows


In [126]:
data_final.head()

,PrimeKey_pbo,SaleMonth,LoanId,Investor_pbosale,PBOSale_SalePassThru,PBOSale_SaleUPB_Orig,GoSSplit_pbosale,MEPeriod_pbo,trade_period,wire_date_remit,...,terminal_path_x_current_upb_x_trade_period,terminal_path_flag_detailed_x_upb_x_trade_period,remit_expected_claims_interest,remit_total_due_investor,terminal_total_prin_minus_origtradeupb,penny_prin_from_shortfall,penny_debrate_savings,remit_interest_paid_prior_periods,total_cashflows_investor,total_cashflows_penny
0,10001549042203,202203,1000154904,NB,0.04,176416.66,0.5,202202,0,2022-03-15,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-179201.98,433.1
1,10001549042203,202203,1000154904,NB,0.04,176416.66,0.5,202203,1,2022-04-15,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
2,10001549042203,202203,1000154904,NB,0.04,176416.66,0.5,202204,2,2022-05-15,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
3,10001549042203,202203,1000154904,NB,0.04,176416.66,0.5,202205,3,2022-06-15,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
4,10001549042203,202203,1000154904,NB,0.04,176416.66,0.5,202206,4,2022-07-15,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0


In [127]:
# print(new_df.loc[new_df['remit_default_dt'] !=None]['remit_default_dt'].head()) # still printed Nones
# print(new_df.loc[new_df['remit_default_dt'].notnull() ]['remit_default_dt'].head()) # this appears to work

In [128]:
data_final.shape

(13842, 183)

## Export Results

In [129]:
data_final.to_csv(f'C:\\Users\\{login}\\Desktop\\data_final_{date_string}_202203SaleMonth.csv', index=True, index_label='Index')

In [135]:
print(data_final[['remit_days_split_interest', 'remit_days_default_dt', 'PBOSale_SaleDate', 'wire_date_remit', 'repurchase_date_calc']].dtypes)
# print(new_df[['remit_days_split_interest', 'remit_days_default_dt']].dtypes)

remit_days_split_interest    float64
remit_days_default_dt        float64
PBOSale_SaleDate              object
wire_date_remit               object
repurchase_date_calc          object
dtype: object


In [123]:
# check dtypes of final calc columns
# print(data_final[['orig_settle_fs_investor_interest', 'orig_settle_fs_investor_principal', 'investor_redelivery_premium_dlr', 'repurchase_interest', 'ia_total_SSEPrincipal', 'ia_total_SSENetinterest', 'redelivery_fee', 'investor_repurch_par_dlr', 'remit_total_due_investor']].dtypes)
print(data_final[['remit_default_dt', 'remit_MinTransactiondate', 'remit_MaxTransactiondate', 'remit_eom_TransactionDate']].dtypes)

remit_default_dt             object
remit_MinTransactiondate     object
remit_MaxTransactiondate     object
remit_eom_TransactionDate    object
dtype: object


In [ ]:
# check dtypes of final calc columns
# print(data_final[['orig_settle_fs_investor_interest', 'orig_settle_fs_investor_principal', 'investor_redelivery_premium_dlr', 'repurchase_interest', 'ia_total_SSEPrincipal', 'ia_total_SSENetinterest', 'redelivery_fee', 'investor_repurch_par_dlr', 'remit_total_due_investor']].dtypes)
print(data_final[['orig_settle_fs_investor_interest', 'orig_settle_fs_investor_principal', 'investor_redelivery_premium_dlr', 'repurchase_interest', 'ia_total_SSEPrincipal', 'ia_total_SSENetinterest', 'remit_expected_claims_interest', 'remit_expected_claims_upb', 'remit_total_due_investor', 'FHA_DebentureRt']].dtypes)

orig_settle_fs_investor_interest     float64
orig_settle_fs_investor_principal    float64
investor_redelivery_premium_dlr      float64
repurchase_interest                  float64
ia_total_SSEPrincipal                float64
ia_total_SSENetinterest              float64
remit_expected_claims_interest       float64
remit_expected_claims_upb            float64
remit_total_due_investor             float64
FHA_DebentureRt                      float64
dtype: object


In [280]:
# check dtypes of penny_debrate_savings calc columns
# print(data_final[['path_final', 'terminal_path_flag', 'LoanType', 'CurrentInterestRate', 'FHA_DebentureRt', 'remit_days_split_interest', 'remit_days_default_dt']].dtypes)

path_final                    object
terminal_path_flag             int64
LoanType                      object
CurrentInterestRate          float64
FHA_DebentureRt              float64
remit_days_split_interest    float64
remit_days_default_dt        float64
dtype: object


## temp Data Final  (grab, fillna, remove unnamed, etc)

In [130]:
data_final = pd.read_csv(r'C:\Users\jboyce\Desktop\data_filtered_202101SaleMonth_20240719_v0_calcd fields_filtered periods.csv')

C:\Users\jboyce\AppData\Local\Temp\ipykernel_15016\762277099.py:1: DtypeWarning: Columns (39,40,76) have mixed types. Specify dtype option on import or set low_memory=False.
  data_final = pd.read_csv(r'C:\Users\jboyce\Desktop\data_filtered_202101SaleMonth_20240719_v0_calcd fields_filtered periods.csv')


In [131]:
data_final=data_final.fillna(0)

In [132]:
data_final = data_final.loc[:, ~data_final.columns.str.contains('^Unnamed')]

In [133]:
data_final = data_final.drop(columns=['Index'])

In [134]:
data_final.head()

,PrimeKey_pbo,SaleMonth,LoanId,Investor_pbosale,PBOSale_SalePassThru,PBOSale_SaleUPB_Orig,GoSSplit_pbosale,MEPeriod_pbo,trade_period,InvestorId,...,orig_settle_fs_penny,redelivery_fee,investor_repurch_par_dlr,repurchase_interest_days,repurchase_interest,remit_expected_claims_interest,remit_total_due_investor,path_final,total_cashflows_investor,total_cashflows_penny
0,10001825542101,202101,1000182554,MM,0.05,88870.46,0.0,202012,0,440,...,-1485.29,0,0.0,0,0.0,0.0,0.0,PIF,-89273.67,-917.16
1,10001825542101,202101,1000182554,MM,0.05,88870.46,0.0,202101,1,D01,...,-0.00,0,0.0,0,0.0,0.0,0.0,PIF,0.00,586.64
2,10001825542101,202101,1000182554,MM,0.05,88870.46,0.0,202102,2,D01,...,-0.00,0,0.0,0,0.0,0.0,0.0,PIF,0.00,586.64
3,10001825542101,202101,1000182554,MM,0.05,88870.46,0.0,202103,3,D01,...,-0.00,0,0.0,0,0.0,0.0,0.0,PIF,0.00,586.64
4,10001825542101,202101,1000182554,MM,0.05,88870.46,0.0,202104,4,D01,...,-0.00,0,0.0,0,0.0,0.0,0.0,PIF,0.00,586.64


In [315]:
temp_df_prep.to_csv(r'C:\Users\jboyce\Desktop\quicktest_20240718_calcd fields_removed periods.csv', index=True, index_label='Index')

In [316]:
temp_df_prep_barings = temp_df_prep.loc[temp_df_prep['Investor_pbosale'].isin(['TO', 'MM'])]

In [317]:
temp_df_prep_barings.to_csv(r'C:\Users\jboyce\Desktop\quicktest_20240718_calcd fields_barings.csv', index=True, index_label='Index')

In [157]:
test1 = new_df.loc[(new_df['LoanId'].isin([1007048586, 1000432405]))]


In [159]:
test1.to_excel(r'C:\Users\jboyce\Desktop\quicktest.xlsx', index=True, index_label='Index')

In [160]:
test1['CurrentPrincipalBalanceAmt_next'].dtype

dtype('float64')

In [199]:
temp_df.head()

,PrimeKey_pbo,SaleMonth,LoanId,Investor_pbosale,PBOSale_SalePassThru,PBOSale_SaleUPB_Orig,GoSSplit_pbosale,MEPeriod_pbo,trade_period,InvestorId,...,penny_redelivery_premium_dlr,Pool Num,Management,Pool Status,Type Dimension,Loan Type,settled date,settled price,EBO_Investor_y,Path_Reperf
0,10001825542101,202101,1000182554,MM,0.05,88870.46,NaN,202012,0,440,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10001825542101,202101,1000182554,MM,0.05,88870.46,NaN,202101,1,D01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10001825542101,202101,1000182554,MM,0.05,88870.46,NaN,202102,2,D01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10001825542101,202101,1000182554,MM,0.05,88870.46,NaN,202103,3,D01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10001825542101,202101,1000182554,MM,0.05,88870.46,NaN,202104,4,D01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [221]:
# temp_df['remit_expected_claims_interest'] = 0
# temp_df.shape
temp_df.head()

,PrimeKey_pbo,SaleMonth,LoanId,Investor_pbosale,PBOSale_SalePassThru,PBOSale_SaleUPB_Orig,GoSSplit_pbosale,MEPeriod_pbo,trade_period,InvestorId,...,Management,Pool Status,Type Dimension,Loan Type,settled date,settled price,EBO_Investor_y,Path_Reperf,remit_expected_claims_interest,test_column
0,10001825542101,202101,1000182554,MM,0.05,88870.46,NaN,202012,0,440,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
1,10001825542101,202101,1000182554,MM,0.05,88870.46,NaN,202101,1,D01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
2,10001825542101,202101,1000182554,MM,0.05,88870.46,NaN,202102,2,D01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
3,10001825542101,202101,1000182554,MM,0.05,88870.46,NaN,202103,3,D01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
4,10001825542101,202101,1000182554,MM,0.05,88870.46,NaN,202104,4,D01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1


In [205]:
temp_df['CurrentPrincipalBalanceAmt_next'].isnull().any()

False

In [169]:
print(new_df[['CurrentPrincipalBalanceAmt_next', 'FHA_DebentureRt', 'CurrentInterestRate', 'LoanType', 'LoanSubStatusId']].dtypes)

CurrentPrincipalBalanceAmt_next    float64
FHA_DebentureRt                    float64
CurrentInterestRate                float64
LoanType                            object
LoanType                            object
LoanSubStatusId                     object
dtype: object


In [184]:
unexpected_chars_filter = new_df['LoanSubStatusId'].astype(str).str.contains(r'[^A-Za-z]')  # Check for non-alphabet chars
problematic_rows = new_df[unexpected_chars_filter][['LoanSubStatusId']].values #  .unique()
print(problematic_rows)


[['D4']
 ['D4']
 ['D4']
 ...
 ['D4']
 ['D3']
 ['D1']]


In [40]:
# df_20220201_TO_trimmed.head()
# data_test.head()

In [41]:
# data_test['PBOSale_SaleDate'].unique()

In [42]:
# data_test.to_csv(r'C:\Users\jboyce\Desktop\data_202101SaleMonth_20240717_v0_quicktest.csv', index=True, index_label='Index')

In [43]:
# df_barings_20240717_v0_quicktest = data_test.loc[(data_test.Investor_pbosale.isin(['TO', 'MM'])) ]

In [44]:
# df_barings_20240717_v0_quicktest.to_csv(r'C:\Users\jboyce\Desktop\data_202101SaleMonth_20240717_v0_quicktest_barings.csv', index=True, index_label='Index')

In [45]:
# ok try to remove extra periods
# data_filtered = prep_all_cashflows(new_df)

In [46]:
# data_filtered.to_csv(r'C:\Users\jboyce\Desktop\data_filtered_202101SaleMonth_20240717_v0.csv', index=True, index_label='Index')

In [47]:
# df_barings_20240717_v1_filtered = data_filtered.loc[(data_filtered.Investor_pbosale.isin(['TO', 'MM'])) ]


In [48]:
# df_barings_20240717_v1_filtered.to_csv(r'C:\Users\jboyce\Desktop\data_filtered_202101SaleMonth_20240717_v1_barings.csv', index=True, index_label='Index')

## PENDING create liquidation tie-out export with remit_fields

## Export Results 2

In [336]:
# add loop where it exports by investor, year or path
# df_20200102_MM_trimmed.to_excel(r'C:\Users\jboyce\Desktop\df_20200102_MM_trimmed.xlsx')

In [30]:
# df_20240710_test = new_df_filtered.loc[(new_df_filtered.LoanId.isin([8031259155, 8031351290, 8027140546, 8023525068, 7001712644, 7001511540, 1005042403, 1004919118])) ]
# data_final_20240714 = data_test.loc[(new_df_filtered.LoanId.isin([8031259155, 8031351290, 8027140546, 8023525068, 7001712644, 7001511540, 1005042403, 1004919118])) ]

In [208]:
df_barings_20240714.to_csv(r'C:\Users\jboyce\Desktop\df_barings_20240714_index.csv', index=True)

## JUNK

In [180]:
# new_df.groupby(by=['PrimeKey'])['trade_period'].count()
new_df_flat_primekey = pd.DataFrame({'count': new_df.groupby(by=['PrimeKey'])['trade_period'].size()}).reset_index()

In [181]:
new_df_flat_primekey.head()

,PrimeKey,count
0,10000338631801,197
1,10000338631904,140
2,10000341732109,14
3,10000342241806,53
4,10000346582106,17


In [183]:
new_df_flat_primekey.loc[new_df_flat_primekey['count'] == 0]

,PrimeKey,count
